Download last year data and input into Data Lake + DB

In [1]:
import requests
import json
import os
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# 1. Konfiguracja
CURRENCIES = ['THB', 'USD', 'AUD', 'HKD', 'CAD', 'NZD', 'SGD', 'EUR', 'CHF', 'HUF', 
              'GBP', 'UAH', 'JPY', 'CZK', 'DKK', 'ISK', 'NOK', 'SEK', 'RON', 'TRY', 
              'CLP', 'PHP', 'MXN', 'ZAR', 'BRL', 'MYR', 'IDR', 'INR', 'KRW', 'CNY']

BRONZE_PATH = Path("/home/jovyan/work/data/bronze/history_2025")
BRONZE_PATH.mkdir(parents=True, exist_ok=True)

spark = SparkSession.builder \
    .appName("NBP_Historical_Ingest") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0") \
    .getOrCreate()

# 2. Pobieranie danych (Bronze Layer)
print(" Rozpoczynam pobieranie historii z NBP...")

for symbol in CURRENCIES:
    try:
        url = f"http://api.nbp.pl/api/exchangerates/rates/a/{symbol}/2025-01-01/2025-12-31/?format=json"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            # Zapis surowego JSONa
            with open(BRONZE_PATH / f"{symbol}_2025.json", 'w') as f:
                json.dump(data, f)
            print(f" Pobrano: {symbol}")
        else:
            print(f"⚠ Pominąłem {symbol} (Status: {response.status_code})")
    except Exception as e:
        print(f" Błąd przy {symbol}: {e}")

# 3. Przetwarzanie Sparkiem (Silver Layer)
print("\n Przetwarzam dane Sparkiem...")

# Wczytujemy wszystkie JSONy naraz z folderu
raw_df = spark.read.option("multiLine", "true").json(f"{BRONZE_PATH}/*.json")

# NBP zwraca dane w formacie: {code: 'USD', rates: [{effectiveDate: '...', mid: ...}, ...]}
# Musimy "rozbić" (explode) listę rates na osobne wiersze
exploded_df = raw_df.select("code", F.explode("rates").alias("rate_data"))

final_df = exploded_df.select(
    F.col("code").alias("currency_code"),
    F.col("rate_data.effectiveDate").cast("date").alias("exchange_date"),
    F.col("rate_data.mid").cast("decimal(10,4)").alias("rate_value")
)

# 4. Zapis do Postgresa
print(" Zapisuję do Postgresa (tabela: f_currency_rates)...")

final_df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_dw:5432/currency_db") \
    .option("dbtable", "f_currency_rates") \
    .option("user", "admin") \
    .option("password", "password123") \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("\n✨ GOTOWE! Twoja historia walut na rok 2025 jest w bazie.")
final_df.groupBy("currency_code").count().show(5)

🚀 Rozpoczynam pobieranie historii z NBP...
✅ Pobrano: THB
✅ Pobrano: USD
✅ Pobrano: AUD
✅ Pobrano: HKD
✅ Pobrano: CAD
✅ Pobrano: NZD
✅ Pobrano: SGD
✅ Pobrano: EUR
✅ Pobrano: CHF
✅ Pobrano: HUF
✅ Pobrano: GBP
✅ Pobrano: UAH
✅ Pobrano: JPY
✅ Pobrano: CZK
✅ Pobrano: DKK
✅ Pobrano: ISK
✅ Pobrano: NOK
✅ Pobrano: SEK
✅ Pobrano: RON
✅ Pobrano: TRY
✅ Pobrano: CLP
✅ Pobrano: PHP
✅ Pobrano: MXN
✅ Pobrano: ZAR
✅ Pobrano: BRL
✅ Pobrano: MYR
✅ Pobrano: IDR
✅ Pobrano: INR
✅ Pobrano: KRW
✅ Pobrano: CNY

⚙️ Przetwarzam dane Sparkiem...
💾 Zapisuję do Postgresa (tabela: f_currency_rates)...

✨ GOTOWE! Twoja historia walut na rok 2025 jest w bazie.
+-------------+-----+
|currency_code|count|
+-------------+-----+
|          IDR|  251|
|          KRW|  251|
|          INR|  251|
|          ISK|  251|
|          JPY|  251|
+-------------+-----+
only showing top 5 rows



Download data from 2020-2024 

In [2]:
import requests
import json
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# 1. Konfiguracja
CURRENCIES = ['THB', 'USD', 'AUD', 'HKD', 'CAD', 'NZD', 'SGD', 'EUR', 'CHF', 'HUF', 
              'GBP', 'UAH', 'JPY', 'CZK', 'DKK', 'ISK', 'NOK', 'SEK', 'RON', 'TRY', 
              'CLP', 'PHP', 'MXN', 'ZAR', 'BRL', 'MYR', 'IDR', 'INR', 'KRW', 'CNY']

YEARS_TO_FETCH = [2020, 2021, 2022, 2023, 2024]

# Ścieżki do obu folderów (stary 2025 i nowy dla reszty)
PATH_2025 = Path("/home/jovyan/work/data/bronze/history_2025")
PATH_PRE_2025 = Path("/home/jovyan/work/data/bronze/history_pre2025")
PATH_PRE_2025.mkdir(parents=True, exist_ok=True)

spark = SparkSession.builder \
    .appName("NBP_Historical_Backfill") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0") \
    .getOrCreate()

# 2. Pobieranie brakujących lat (2020-2024)
print(f" Pobieram brakujące lata {YEARS_TO_FETCH}...")

for year in YEARS_TO_FETCH:
    print(f"--- Rok {year} ---")
    for symbol in CURRENCIES:
        file_name = PATH_PRE_2025 / f"{symbol}_{year}.json"
        if file_name.exists(): continue
            
        try:
            url = f"http://api.nbp.pl/api/exchangerates/rates/a/{symbol}/{year}-01-01/{year}-12-31/?format=json"
            response = requests.get(url)
            if response.status_code == 200:
                with open(file_name, 'w') as f:
                    json.dump(response.json(), f)
            # Mała pauza, żeby API NBP nas nie polubiło za bardzo (opcjonalne)
        except Exception as e:
            print(f" Błąd: {symbol} {year}: {e}")

# 3. Przetwarzanie Sparkiem - Łączymy oba źródła!
print("\n Spark łączy dane z lat 2020-2025...")

# Spark potrafi czytać z wielu ścieżek naraz
all_files = [f"{PATH_2025}/*.json", f"{PATH_PRE_2025}/*.json"]
raw_df = spark.read.option("multiLine", "true").json(all_files)

exploded_df = raw_df.select("code", F.explode("rates").alias("rate_data"))

final_df = exploded_df.select(
    F.col("code").alias("currency_code"),
    F.col("rate_data.effectiveDate").cast("date").alias("exchange_date"),
    F.col("rate_data.mid").cast("decimal(10,4)").alias("rate_value")
).distinct()

# 4. Zapis do Postgresa (Overwrite - robimy świeżą, pełną tabelę)
print(f" Zapisuję pełną historię ({final_df.count()} rekordów) do bazy...")

final_df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_dw:5432/currency_db") \
    .option("dbtable", "f_currency_rates") \
    .option("user", "admin") \
    .option("password", "password123") \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("\n✨ SUKCES! Twoja baza jest teraz pełna danych od 2020 do 2025.")

🚀 Pobieram brakujące lata [2020, 2021, 2022, 2023, 2024]...
--- Rok 2020 ---
--- Rok 2021 ---
--- Rok 2022 ---
--- Rok 2023 ---
--- Rok 2024 ---

⚙️ Spark łączy dane z lat 2020-2025...
💾 Zapisuję pełną historię (45450 rekordów) do bazy...

✨ SUKCES! Twoja baza jest teraz pełna danych od 2020 do 2025.


Download historial XAU prices from 2020-2025

In [3]:
import requests
import json
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# 1. Ustawienia ścieżek
GOLD_BRONZE_PATH = Path("/home/jovyan/work/data/bronze/historical_data_gold")
GOLD_BRONZE_PATH.mkdir(parents=True, exist_ok=True)

spark = SparkSession.builder \
    .appName("NBP_Gold_Backfill") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0") \
    .getOrCreate()

# 2. Pobieranie danych 2020-2025
YEARS = [2020, 2021, 2022, 2023, 2024, 2025]
print(" Rozpoczynam pobieranie cen złota...")

for year in YEARS:
    file_path = GOLD_BRONZE_PATH / f"gold_{year}.json"
    if not file_path.exists():
        url = f"http://api.nbp.pl/api/cenyzlota/{year}-01-01/{year}-12-31/?format=json"
        res = requests.get(url)
        if res.status_code == 200:
            with open(file_path, 'w') as f:
                json.dump(res.json(), f)
            print(f"✅ Pobrano rok {year}")

# 3. Spark: Transformacja do formatu Silver
print("\n Spark przetwarza pliki JSON...")
gold_df = spark.read.option("multiLine", "true").json(f"{GOLD_BRONZE_PATH}/*.json")

# Dostosowujemy do struktury naszej tabeli w Postgresie
gold_final = gold_df.select(
    F.lit("XAU").alias("currency_code"),
    F.col("data").cast("date").alias("exchange_date"),
    F.col("cena").cast("decimal(10,2)").alias("rate_value")
)

# 4. Zapis do Postgresa (używamy 'append', bo tabela już istnieje)
print(" Wsiadam do Postgresa...")
gold_final.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_dw:5432/currency_db") \
    .option("dbtable", "f_gold_prices") \
    .option("user", "admin") \
    .option("password", "password123") \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()

print("\n ZŁOTO ZAŁADOWANE!")

🟡 Rozpoczynam pobieranie cen złota...
✅ Pobrano rok 2020
✅ Pobrano rok 2021
✅ Pobrano rok 2022
✅ Pobrano rok 2023
✅ Pobrano rok 2024
✅ Pobrano rok 2025

⚙️ Spark przetwarza pliki JSON...
💾 Wsiadam do Postgresa...

✨ ZŁOTO ZAŁADOWANE!


Download spreads for calculations


In [4]:
import requests
import json
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# 1. Konfiguracja
SPREAD_CURRENCIES = ['USD', 'EUR', 'CHF', 'GBP', 'CZK']
YEARS = [2020, 2021, 2022, 2023, 2024, 2025]
SPREAD_PATH = Path("/home/jovyan/work/data/bronze/historical_data_spread")
SPREAD_PATH.mkdir(parents=True, exist_ok=True)

spark = SparkSession.builder \
    .appName("NBP_Spread_Ingest") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0") \
    .getOrCreate()

# 2. Pobieranie danych (Bronze)
print(" Pobieram dane z Tabeli C (Kupno/Sprzedaż)...")
for year in YEARS:
    for symbol in SPREAD_CURRENCIES:
        file_name = SPREAD_PATH / f"spread_{symbol}_{year}.json"
        if file_name.exists(): continue
        
        try:
            url = f"http://api.nbp.pl/api/exchangerates/rates/c/{symbol}/{year}-01-01/{year}-12-31/?format=json"
            res = requests.get(url)
            if res.status_code == 200:
                with open(file_name, 'w') as f:
                    json.dump(res.json(), f)
        except Exception as e:
            print(f"❌ Błąd {symbol} {year}: {e}")

# 3. Przetwarzanie Sparkiem (Silver) - Obliczanie Spreadu
print("\n Spark oblicza spready...")
raw_spread_df = spark.read.option("multiLine", "true").json(f"{SPREAD_PATH}/*.json")

exploded_spread_df = raw_spread_df.select("code", F.explode("rates").alias("r"))

final_spread_df = exploded_spread_df.select(
    F.col("code").alias("currency_code"),
    F.col("r.effectiveDate").cast("date").alias("exchange_date"),
    F.col("r.bid").cast("decimal(10,4)").alias("bid_price"),
    F.col("r.ask").cast("decimal(10,4)").alias("ask_price")
).withColumn("spread_value", F.round(F.col("ask_price") - F.col("bid_price"), 4))

# 4. Zapis do Postgresa
print(" Zapisuję do tabeli f_currency_spreads...")
final_spread_df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_dw:5432/currency_db") \
    .option("dbtable", "f_currency_spreads") \
    .option("user", "admin") \
    .option("password", "password123") \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()

print("\n GOTOWE! Spready załadowane.")
final_spread_df.show(5)

🚀 Pobieram dane z Tabeli C (Kupno/Sprzedaż)...

⚙️ Spark oblicza spready...
💾 Zapisuję do tabeli f_currency_spreads...

✨ GOTOWE! Spready załadowane.
+-------------+-------------+---------+---------+------------+
|currency_code|exchange_date|bid_price|ask_price|spread_value|
+-------------+-------------+---------+---------+------------+
|          USD|   2020-01-02|   3.7597|   3.8357|      0.0760|
|          USD|   2020-01-03|   3.7667|   3.8427|      0.0760|
|          USD|   2020-01-07|   3.7679|   3.8441|      0.0762|
|          USD|   2020-01-08|   3.7704|   3.8466|      0.0762|
|          USD|   2020-01-09|   3.7724|   3.8486|      0.0762|
+-------------+-------------+---------+---------+------------+
only showing top 5 rows

